In [ ]:
!pip install PyMuPDF

In [ ]:
!pip install qdrant-client sentence-transformers

In [ ]:
# Import library
from qdrant_client import QdrantClient
from qdrant_client.models import PointStruct, Distance, VectorParams, FieldCondition, MatchValue
from sentence_transformers import SentenceTransformer
import fitz

# Inisialisasi embedding model (Sentence Transformers)
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

# Fungsi untuk mengambil kalimat dari PDF
def extract_sentences_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)  # Membuka file PDF
    sentences = []

    # Menyusuri setiap halaman dalam PDF
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)  # Memuat halaman ke-`page_num`
        text = page.get_text("text")  # Ekstrak teks dalam bentuk string
        page_sentences = text.split('.')  # Pisahkan teks menjadi kalimat-kalimat berdasarkan tanda titik

        # Membersihkan dan menambahkan kalimat yang valid
        for sentence in page_sentences:
            clean_sentence = sentence.strip()
            if clean_sentence:
                sentences.append(clean_sentence)

    return sentences

# Path ke file PDF
pdf_path = "/content/LSPI_T3_2024.pdf"

# Ambil kalimat-kalimat dari PDF
sentences = extract_sentences_from_pdf(pdf_path)

# Generate embeddings untuk kalimat-kalimat yang diambil
embeddings = embedding_model.encode(sentences)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
# Connect ke Qdrant
client = QdrantClient(":memory:")
collection_name = "lspi_embeddings"

# Buat sebuah collection di Qdrant
client.recreate_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(size=len(embeddings[0]), distance=Distance.COSINE),
)

# Masukkan embedding ke dalam Qdrant
points = [
    PointStruct(
        id=i,
        vector=embeddings[i],
        payload={"sentence": sentences[i]}
    )
    for i in range(len(sentences))
]

# Menambahkan embedding ke koleksi Qdrant
client.upsert(collection_name=collection_name, points=points)

<ipython-input-5-e201a00e3502>:6: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [ ]:
def query_product_info(query, collection_name):
    # Generate embedding untuk query pengguna
    query_embedding = embedding_model.encode(query)

    # Mencari kalimat yang paling relevan di Qdrant berdasarkan vektor query
    search_result = client.search(
        collection_name=collection_name,
        query_vector=query_embedding.tolist(),
        limit=1,  # Mengambil satu hasil paling relevan
    )

    # Mengambil kalimat dari hasil pencarian
    most_similar_sentence = search_result[0].payload['sentence']
    similarity_score = search_result[0].score

    return most_similar_sentence, similarity_score

In [ ]:
# Pertanyaan dari pengguna (query)
user_query = "Bagaimana pertumbuhan ekonomi Indonesia pada triwulan III-2024 dari sisi domestik?"

# Menjalankan pencarian berdasarkan query
answer, score = query_product_info(user_query, collection_name)

# Menampilkan hasil
print(f"Jawaban untuk query '{user_query}':")
print(f"Kalimat terkait: {answer}")
print(f"Skor kemiripan: {score:.4f}")

Jawaban untuk query 'Bagaimana pertumbuhan ekonomi Indonesia pada triwulan III-2024 dari sisi domestik?':
Kalimat terkait: Perekonomian Domestik 
Dari sisi domestik, pertumbuhan ekonomi 
Indonesia pada triwulan III-2024 tumbuh 
moderat sebesar 4,95% (yoy), relatif masih 
sama dengan triwulan III tahun lalu sebesar 
4,94% (yoy)
Skor kemiripan: 0.8561


<ipython-input-6-728a4cb080a1>:6: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  search_result = client.search(


In [ ]:
context = f"""LAPORAN SURVEILLANCE PERBANKAN INDONESIA Triwulan III 2024 - Overview dan Analisis Perekonomian Global dan Domestik \n\n\n"""

In [ ]:
pip install groq -q

In [ ]:
# Import Library Groq
from groq import Groq
import getpass

# Meminta input API Key dengan getpass
GROQ_API_KEY=getpass.getpass("Groq API Key: ")

# Membuat objek client untuk berinteraksi dengan Groq
client = Groq(api_key=GROQ_API_KEY)

Groq API Key: ··········


In [ ]:
system_message = f"""
You're a helpful assistant for Laporan Surveillance Perbankan Indonesia.
You're going to check Overview dan Analisis Perekonomian Global dan Domestik.
Your answer is always brief and direct.
"""

In [ ]:
# Buat fungsi yang menerima dua parameter query dan context
def get_response(query: str, context: str):
  completion = client.chat.completions.create(
      model="llama-3.2-90b-vision-preview",
      messages=[
          {
              "role": "system",
              "content": system_message
          },
          {
              "role": "user",
              "content": context
          },
          {
              "role": "user",
              "content": query
          }
      ],
      temperature=0,
      max_tokens=1024,
      top_p=1,
      stream=False,
      stop=None,
  )

  return completion.choices[0].message.content

In [ ]:
answer = get_response(
    query="Bagaimana pertumbuhan ekonomi Indonesia pada triwulan III-2024 dari sisi domestik?",
    context=context,
    )

print(answer)

Pertumbuhan ekonomi Indonesia pada triwulan III-2024 dari sisi domestik menunjukkan peningkatan sebesar 5,3% YoY, didorong oleh konsumsi rumah tangga dan investasi yang kuat.


In [ ]:
print(
    get_response(
    query="Bagaiman kondisi ekonomi global pada tahun 2024?",
    context=context,
    )
)

Pada Triwulan III 2024, kondisi ekonomi global masih menghadapi tantangan, termasuk inflasi yang tinggi, ketidakpastian geopolitik, dan perlambatan pertumbuhan ekonomi di beberapa negara maju. Namun, beberapa negara berkembang seperti Asia Tenggara masih menunjukkan pertumbuhan ekonomi yang relatif stabil.


In [ ]:
!pip install gradio openai

In [ ]:
# Import library gradio
import gradio as gr

# Fungsi untuk menangani input query yang diberikan oleh pengguna
def chat_interface(query, context):
    try:
        response = get_response(query, context)
        return response
    except Exception as e:
        return f"Error: {str(e)}"

# Membuat antarmuka (Gradio Interface)
iface = gr.Interface(
    fn=chat_interface,
    inputs=[
        gr.Textbox(label="Your Query")
    ],
    outputs=gr.Textbox(label="Assistant Response"),
    title="LLM Chat Interface",
    description="Chat with the LLM"
)

# Meluncurkan antarmuka web
iface.launch(pwa=True, share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b0ef431f8829762ee4.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
